In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
train= pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/train.csv')
feat = pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/features.csv')
pnt  = pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv')

# **#Basic EDA of each dataset**

In this notebook,I had done some eda on each data set to understand the data

# **patients data**

**patient_notes.csv - A collection of about 40,000 Patient Note history portions. Only a subset of these have features annotated. You may wish to apply unsupervised learning techniques on the notes without annotations. The patient notes in the test set are not included in the public version of this file.**

    pn_num - A unique identifier for each patient note.
    case_num - A unique identifier for the clinical case a patient note represents.
    pn_history - The text of the encounter as recorded by the test taker.

In [ ]:
pnt.head()

In [ ]:
print('No of rows in patient note data:',pnt.shape)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='case_num',data=pnt)
plt.title('No of clinical cases present in data')
plt.xticks(range(10),['Case_'+str(i) for i in range(10)])

plt.show()

In [ ]:
pnt['pn_history'].sample().values[0]

In [ ]:
note_len=pnt['pn_history'].values

lent=[len(i) for i in note_len]


sns.distplot(lent,color='red')
plt.xlabel('Length of patient notes')
plt.show()

In [ ]:
print('avergae length of patient notes',np.mean(lent))
print('Minimum  length of patient notes:',min(lent))
print('Maximum length of patient notes:',max(lent))

print()
print('99 percentage of data has length of patient notes:',np.percentile(lent,99))
print('98 percentage of data has length of patient notes:',np.percentile(lent,98))
print('90 percentage of data has length of patient notes:',np.percentile(lent,90))

In [ ]:
def gender(text):
    s=text.lower().split()
    for i in s:
        if i in ['male','m','he','mr']:
            return 'male'
            break

        elif i in ['female','f','she','ms']:
            return 'female'
            break
    else:
        return None

In [ ]:
pnt['gender']=pnt['pn_history'].apply(lambda x:gender(x))


sns.countplot(x='gender',data=pnt)
plt.title('gender of all patients who came for treatment')
plt.show()

In [ ]:
pnt['start_few_words']=pnt['pn_history'].apply(lambda x: x.split()[:10])
pnt.head()

In [ ]:
year=[]
for i in pnt['start_few_words'].values:
    for j in i:
        if ('y' in j) & (j[0]=='y'):
            year.append(j)
            
year=np.unique(year)
year=[i for i in year if len(i)<=8]

print(year)

In [ ]:
age=[]
for i in pnt['start_few_words'].values:
    for ind,j in enumerate(i):
        if j in year:
            age.append(i[ind-1])


age=[i for i in age if i.isdigit()]

age=pd.DataFrame(data=age,columns=['age'])


In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='age',data=age,order=age['age'].value_counts().sort_values(ascending=False).index[:15])
plt.title('age group present in data i.e who came for visit doctor')
plt.show()

# **Feature data**

**features.csv - The rubric of features (or key concepts) for each clinical case.**

    feature_num - A unique identifier for each feature.
    case_num - A unique identifier for each case.
    feature_text - A description of the feature.

In [ ]:
feat.head()

In [ ]:
print('No of rows in feature data:',feat.shape)

In [ ]:
sns.countplot(x='case_num',data=feat,order=feat['case_num'].value_counts().index)
plt.title('No of clinical cases present in featue data')
plt.show()

In [ ]:
feat['feature_text'].sample().values[0].split('-')

In [ ]:
def feature_words(type):

    all_feat=[]
    
    if type=='individual':
        for i in feat['feature_text']:
            all_feat.append(i.split('-')[0])
        all_feat=pd.DataFrame(data=all_feat,columns=['all_feat'])
        
        plt.figure(figsize=(10,6))
        sns.countplot(x='all_feat',data=all_feat,order=all_feat['all_feat'].value_counts().sort_values(ascending=False).index[:15])
        plt.title('most repeated inidividual/single words present in feature text')
        plt.xticks(rotation=90)
        plt.show()

    elif type=='raw_features':
        
        for i in feat['feature_text']:
            all_feat.append(i.split()[0])
        all_feat=pd.DataFrame(data=all_feat,columns=['all_feat']) 
        
        plt.figure(figsize=(10,6))
        sns.countplot(x='all_feat',data=all_feat,order=all_feat['all_feat'].value_counts().sort_values(ascending=False).index[:15])
        plt.title('most repeated feature present in feature text')
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
feature_words('individual')

# **train data**

**train.csv - Feature annotations for 1000 of the patient notes, 100 for each of ten cases.**

    id - Unique identifier for each patient note / feature pair.
    pn_num - The patient note annotated in this row.
    feature_num - The feature annotated in this row.
    case_num - The case to which this patient note belongs.
    annotation - The text(s) within a patient note indicating a feature. A feature may be indicated multiple times within a single note
    location - Character spans indicating the location of each annotation within the note. Multiple spans may be needed to represent an 
    annotation, in which case the spans are delimited by a semicolon ;.

In [ ]:
train.head()

In [ ]:
print('No of rows in train:',train.shape)

In [ ]:
sns.countplot(x='case_num',data=train,order=train['case_num'].value_counts().index)
plt.title('No of clinical cases present in train data')
plt.show()

About Annotatios

In [ ]:
print('No of Patinet notes who had feature annotation:',len(train['pn_num'].value_counts()))

In [ ]:
pn_count=train['pn_num'].value_counts()
print(pn_count)

In [ ]:
print('No of Annotations given to different patient notes',np.unique(pn_count.values))

In [ ]:
plt.figure(figsize=(10,5))

sns.countplot(x=train['pn_num'].value_counts().values)
plt.title('No of Patients notes who had same no of annotations')
plt.xticks(rotation=90)
plt.xlabel('Possible no of Annotations present in data i.e gather for each patient note')
plt.show()

In [ ]:
train['location']=train['location'].apply(eval)
train['annotation']=train['annotation'].apply(eval)

In [ ]:
train.head()

In [ ]:
train['annot_count']=train['location'].apply( lambda x:len(x))
train.head()

In [ ]:
#No of annotation count given to each row or multple annotations present in each row
train['annot_count'].value_counts()

**Lets combine all three dataset for one patient note i.e 16 to understand the pattern**

In [ ]:
tr_16=train[train['pn_num']==16]
tr_16.head(15)

In [ ]:
pn_16=pnt[pnt['pn_num']==16]
pn_16.head()

In [ ]:
f=feat[feat['case_num']==0]
f.head(20)

In [ ]:
merge=pd.merge(tr_16,f,on=['feature_num','case_num'],how='inner')
merge.head(20)